In [1]:
# Import utils
import numpy as np
import pandas as pd
import seaborn as sns
import copy
import time
import datetime as dt
import pickle
import pyreadr
import json
import pickle
from joblib import dump, load, Parallel, delayed
import os

**Aggregate results**

In [2]:
def aggregateResults(taus, SKUs, path):
    
    results = pd.DataFrame()
    
    # For all SKUs
    for tau in taus:
        for SKU in SKUs:
            # Check if results exist   
            if os.path.exists(path+'/GwSAA_NEW_SKU'+str(SKU)+'_tau'+str(tau)+'.csv'):
                results = pd.concat([results, pd.read_csv(path+'/GwSAA_NEW_SKU'+str(SKU)+'_tau'+str(tau)+'.csv')])

    return(results)

In [3]:
results_ = aggregateResults(taus=range(0,4+1), SKUs=range(1,460+1), path='/home/fesc/MM/Data/Results/GwSAA_NEW')

In [4]:
results_.groupby('tau').agg({'cpu_time_sec': np.mean}).reset_index()

,tau,cpu_time_sec
0,0,0.448852
1,1,6.801693
2,2,14.792114
3,3,23.583061
4,4,31.550038


In [5]:
results_.groupby(['CR', 'tau']).agg({'cost': sum}).reset_index()

,CR,tau,cost
0,0.50,0,7886867.5
1,0.50,1,8250549.0
2,0.50,2,8245974.5
3,0.50,3,8312738.5
4,0.50,4,8323042.0
5,0.75,0,9782447.5
6,0.75,1,11941609.5
7,0.75,2,12145166.5
8,0.75,3,12195129.0
9,0.75,4,12176231.5


In [107]:
### Function to aggregate all results
def aggregateResults(taus, SKUs, path_to_save, name_to_save):
    
    results = pd.DataFrame()
    
    # For each look-ahead tau=0,...,4
    for tau in taus:
        # For each product (SKU) k=1,...,M
        for SKU in SKUs:
            file_name = path_to_save+'/'+name_to_save+'_SKU'+str(SKU)+'_tau'+str(tau)+'.csv'
            # Check if results exist   
            if os.path.exists(file_name):
                results = pd.concat([results, pd.read_csv(file_name)])

    return(results)

In [108]:
results = aggregateResults(taus=range(0,4+1), SKUs=range(1,460+1), path_to_save='/home/fesc/MM/Data/Results/GwSAAR_NEW', name_to_save='GwSAAR_NEW_e01')

In [109]:
sum(results.defaulted)

0

In [110]:
results.groupby('tau').agg({'cpu_time_sec': np.mean}).reset_index()

,tau,cpu_time_sec
0,0,0.763309
1,1,10.371175
2,2,21.432693
3,3,33.659885
4,4,44.479428


In [111]:
results.groupby(['CR', 'tau']).agg({'cost': sum}).reset_index()

,CR,tau,cost
0,0.50,0,11007347.0
1,0.50,1,10622321.5
2,0.50,2,10563249.5
3,0.50,3,10533568.0
4,0.50,4,10536104.5
5,0.75,0,13507588.5
6,0.75,1,13061670.5
7,0.75,2,13093964.5
8,0.75,3,13009788.0
9,0.75,4,12968235.0


In [112]:
results = pd.DataFrame()

for e in [0.1,1,3,6,12]:
    result = aggregateResults(taus=range(0,4+1), SKUs=range(1,460+1), path_to_save='/home/fesc/MM/Data/Results/GwSAAR_NEW', name_to_save='GwSAAR_NEW_e'+str(e).replace('.', ''))
    results = pd.concat([results, result])

In [113]:
results

,CR,tau,SKU,e,LogToConsole,Threads,NonConvex,PSDTol,MIPGap,NumericFocus,...,I_q_y,c_o,c_s,cost,defaulted,status,solutions,gap,exec_time_sec,cpu_time_sec
0,0.5,0,1,0.1,0,1,2,0.001,0.001,0,...,-366.0,299.5,366.0,665.5,False,2,2,3.386730e-06,1.873937,1.833220
1,0.5,0,1,0.1,0,1,2,0.001,0.001,0,...,-382.0,472.0,382.0,854.0,False,2,2,2.951400e-07,1.358953,1.358210
2,0.5,0,1,0.1,0,1,2,0.001,0.001,0,...,-439.0,477.0,439.0,916.0,False,2,2,5.391556e-07,1.279030,1.278291
3,0.5,0,1,0.1,0,1,2,0.001,0.001,0,...,-570.0,507.5,570.0,1077.5,False,2,2,7.070578e-08,1.275138,1.274421
4,0.5,0,1,0.1,0,1,2,0.001,0.001,0,...,-303.0,579.0,303.0,882.0,False,2,2,2.599698e-10,1.426661,1.425824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,0.9,4,460,12.0,0,1,2,0.001,0.001,0,...,139.0,118.5,139.0,257.5,False,2,2,0.000000e+00,108.831292,105.506364
35,0.9,4,460,12.0,0,1,2,0.001,0.001,0,...,119.0,0.0,119.0,119.0,False,2,4,0.000000e+00,78.100415,77.641443
36,0.9,4,460,12.0,0,1,2,0.001,0.001,0,...,119.0,0.0,119.0,119.0,False,2,4,0.000000e+00,55.027118,50.367270
37,0.9,4,460,12.0,0,1,2,0.001,0.001,0,...,89.0,0.0,89.0,89.0,False,2,2,0.000000e+00,10.441515,10.351318


In [114]:
results_summary = results.groupby(['CR', 'tau', 'e', 'SKU']).agg({'cost': sum}).groupby(['CR', 'SKU']).agg({'cost': min}).reset_index()
results_summary

,CR,SKU,cost
0,0.5,1,8795.5
1,0.5,2,13170.5
2,0.5,3,23854.0
3,0.5,4,1208800.5
4,0.5,5,21273.5
...,...,...,...
1375,0.9,456,2720.0
1376,0.9,457,1479.5
1377,0.9,458,14792.5
1378,0.9,459,569.0


In [115]:
_results_summary = results_.groupby(['CR', 'tau', 'SKU']).agg({'cost': sum}).groupby(['CR', 'SKU']).agg({'cost': min}).reset_index()
_results_summary

,CR,SKU,cost
0,0.5,1,8690.0
1,0.5,2,13205.0
2,0.5,3,23133.0
3,0.5,4,1530609.5
4,0.5,5,21404.5
...,...,...,...
1375,0.9,456,2891.0
1376,0.9,457,1382.0
1377,0.9,458,14727.5
1378,0.9,459,588.0


In [116]:
results_summary.groupby('CR').agg({'cost': sum})

,cost
CR,
0.50,7115760.5
0.75,8927809.0
0.90,10657336.5


In [117]:
_results_summary.groupby('CR').agg({'cost': sum})

,cost
CR,
0.50,7439302.0
0.75,9415701.0
0.90,13177200.0


In [97]:
results_summary.groupby('CR').agg({'cost': np.median})

,cost
CR,
0.50,1414.50
0.75,1882.00
0.90,2252.75


In [98]:
_results_summary.groupby('CR').agg({'cost': np.median})

,cost
CR,
0.50,1445.25
0.75,1849.75
0.90,2136.00


In [118]:
pd.DataFrame(results.groupby(['CR', 'tau', 'e', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).apply(

    lambda r: r.e.iloc[np.argmin(r.cost)]

)).reset_index()[0].value_counts()

0.1     505
1.0     443
3.0     312
6.0      84
12.0     36
Name: 0, dtype: int64

In [ ]:
0.33, 0.66, 1.0, 1.33, 1.66, 2, 2.33, 2.66, 3,

In [ ]:
def aggregateResults(SKU_range, e_range, path):
    
    results = pd.DataFrame()
    
    # For all SKUs
    for SKU in SKU_range:
        
        # For all e
        for e in e_range:

            # Check if results exist   
            if os.path.exists(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e'+str(e)+'.csv'):
                results = pd.concat([results, pd.read_csv(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e'+str(e)+'.csv')])
                
    return(results)

In [ ]:
def aggregateResults(SKU_range, e, path):
    
    results = pd.DataFrame()
    
    # For all SKUs
    for SKU in SKU_range:

        # Check if results exist   
        if os.path.exists(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e'+str(e)+'_UPDATE_FINAL.csv'):
            results = pd.concat([results, pd.read_csv(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e'+str(e)+'_UPDATE_FINAL.csv')])

    return(results)

In [ ]:
PATH_RESULTS = '/home/fesc/MM/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)

# SKU range
e = 12

Results = aggregateResults(SKU_range=SKU_range, e=e, path=PATH_RESULTS)

In [ ]:
len(set(Results.SKU.values))

In [ ]:
sum(Results.defaulted) / len(Results)

In [ ]:
PATH_RESULTS = '/home/fesc/MM/Data/Results/GwSAAR'
SKU_range = range(1,460+1)
missingSKUs = {}
for e in [1,3,6,12]:
    
    Results = aggregateResults(SKU_range=SKU_range, e=e, path=PATH_RESULTS)
    missingSKUs[e] = [i for i in range(1,460+1) if not i in Results.SKU.unique()]

In [ ]:
len(missingSKUs[1]), len(missingSKUs[3]), len(missingSKUs[6]), len(missingSKUs[12])

In [ ]:
missingSKUs[1]

In [ ]:
missingSKUs[3]

In [ ]:
missingSKUs[6]

In [ ]:
missingSKUs[12]

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)

# SKU range
e_range = [1,3,6,12]

Results = aggregateResults(SKU_range=SKU_range, e_range=e_range, path=PATH_RESULTS)
Results

In [ ]:
# Save result
save_log = Results.to_csv(
    path_or_buf=PATH_RESULTS+'/Results_GwSAAR.csv', 
    sep=',', index=False
)

**Check results**

In [ ]:
# Indicator if the period was solved (at least one solution)
Results['periodSolved'] = Results.solutions>0

In [ ]:
# Number of periods solved
ResultsAgg = Results.groupby(['SKU', 'CR', 'e', 'tau']).agg({'periodSolved': sum}).reset_index()

In [ ]:
#### Solved
ResultsSolved = ResultsAgg.loc[ResultsAgg.periodSolved==13]
ResultsSolved

In [ ]:
#### Not solved

# Full grid of results
SKU_range = range(1,460+1)
e_range = [1,3,6,12]
CR_range = [0.5, 0.75, 0.9]
tau_range = [1,2,3,4,5]

Grid = np.array(np.meshgrid(SKU_range, e_range, CR_range, tau_range)).reshape(4,len(SKU_range)*len(e_range)*len(CR_range)*len(tau_range)).T
Grid = pd.DataFrame(Grid, columns=['SKU', 'e', 'CR', 'tau']).astype({'SKU': int, 'e': int, 'tau': int})

# Merge with problem solved
Grid = pd.merge(
    left = Grid, 
    right = ResultsSolved, 
    how = 'outer',
    on = ['SKU', 'e', 'CR', 'tau']
)

# Result: problem not solved
ResultsNotSolved = Grid.loc[np.isnan(Grid.periodSolved)]
ResultsNotSolved

In [ ]:
ResultsNotSolved.loc[ResultsNotSolved.tau <= 2]

In [ ]:
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################
################################################################################################################################

In [ ]:
results_agg['solFound'] = results_agg.solFound == 13
results_agg = results_agg.groupby(['SKU', 'CR']).agg({'solFound': sum}).reset_index()
results_agg['solFound'] = results_agg.solFound > 0

In [ ]:
results.columns

In [ ]:
results['solFound'] = (results.status == 2) + (results.status == 11)
results_agg = results.groupby(['SKU', 'CR', 'tau']).agg({'solFound': sum}).reset_index()

In [ ]:
results_agg['solFound'] = results_agg.solFound == 13
results_agg = results_agg.groupby(['SKU', 'CR']).agg({'solFound': sum}).reset_index()
results_agg['solFound'] = results_agg.solFound > 0

In [ ]:
results['solved'] = np.isnan(results.solutions) + (results.solutions==0)

In [ ]:
results['solved'] = results.solutions>0

In [ ]:
results.loc[np.isnan(results.solutions)]

In [ ]:
len(results)

In [ ]:
sum(np.isnan(results.solutions) + (results.solutions==0))

In [ ]:
sum(results.solutions>0)

In [ ]:
sum(np.isnan(results.solutions))

In [ ]:
sum(results.solutions==0)

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAA'

# SKU range
SKU_range = range(1,460+1)

results = pd.DataFrame()

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    if os.path.exists(PATH_RESULTS+'/GwSAA_SKU'+str(SKU)+'.csv'):
        results = results.append(pd.read_csv(PATH_RESULTS+'/GwSAA_SKU'+str(SKU)+'.csv'))

In [ ]:
sum(results.solutions>0) == len(results)

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)

results = pd.DataFrame()

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    if os.path.exists(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv'):
        results = results.append(pd.read_csv(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv'))

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)

fileExists = []
SKUs = []

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    fileExists = fileExists + [os.path.exists(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv')]
    SKUs = SKUs + [SKU]
        
        
        
ResultsCheck = pd.DataFrame({
    'SKU': SKUs,
    'fileExists': fileExists
})

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)

fileExists = []
SKUs = []

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    fileExists = fileExists + [os.path.exists(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv')]
    SKUs = SKUs + [SKU]
        
        
        
ResultsCheck = pd.DataFrame({
    'SKU': SKUs,
    'fileExists': fileExists
})

In [ ]:
sum(ResultsCheck.fileExists)

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)

results = pd.DataFrame()

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    if os.path.exists(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv'):
        results = results.append(pd.read_csv(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv'))

In [ ]:
sum(results.status == 2) + sum(results.status == 11), len(results.status)

In [ ]:
results['solFound'] = (results.status == 2) + (results.status == 11)
results_agg = results.groupby(['SKU', 'CR', 'tau']).agg({'solFound': sum}).reset_index()

In [ ]:
results_agg['solFound'] = results_agg.solFound == 13
results_agg = results_agg.groupby(['SKU', 'CR']).agg({'solFound': sum}).reset_index()
results_agg['solFound'] = results_agg.solFound > 0

In [ ]:
sum(results_agg.solFound)

In [ ]:
len(set(results.SKU)) * 3

In [ ]:
results_agg = results_agg.groupby(['SKU']).agg({'solFound': sum}).reset_index()
results_agg['solFound'] = results_agg.solFound == 3

In [ ]:
sum(results_agg.solFound)

**Aggregating results**

Aggregating results per product (SKU), CR, and tau...

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)

results = pd.DataFrame()

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    if os.path.exists(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv'):
        results = results.append(pd.read_csv(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e12.csv'))

In [ ]:
results

In [ ]:
# Save result
save_log = results.to_csv(
    path_or_buf=PATH_RESULTS+'/Results_GwSAAR_e12.csv', 
    sep=',', index=False
)

In [ ]:
# Save result
save_log = results.to_csv(
    path_or_buf=PATH_RESULTS+'/Results_GwSAA.csv', 
    sep=',', index=False
)

In [ ]:
PATH_RESULTS

**Archive**

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)


# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    results = pd.read_csv(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e3.csv')
    results = results.drop(columns='Unnamed: 0')
    # Save result
    save_log = results.to_csv(
        path_or_buf=PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e3.csv', 
        sep=',', index=False
    )

    

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results/GwSAAR'

# SKU range
SKU_range = range(1,460+1)


# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    results = pd.read_csv(PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e3.csv')
    results = results.rename(columns={'SL': 'CR'})
    
    # Save result
    save_log = results.to_csv(
        path_or_buf=PATH_RESULTS+'/GwSAAR_SKU'+str(SKU)+'_e3.csv', 
        sep=',', index=False
    )

    

In [ ]:
#results = results.drop(columns='Unnamed: 0')
results.rename(columns={'SL': 'CR'})

In [ ]:
results

In [ ]:
# Save result
save_log = results.to_csv(
    path_or_buf=PATH_RESULTS+'/___GwSAAR_SKU'+str(SKU)+'_e1.csv', 
    sep=',', index=False
)


In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results'
model_type = 'WeightedSAA' # Use 'SAA' or 'WeightedSAA'
sampling = 'global' # Use 'global' or 'local'
scale = True # Use 'global' or 'local'
if scale:
    z='_z'
else:
    z=''

SKU_range = range(1,460+1)
#RO_range = [1,3,6,12,24,48]

results = pd.DataFrame()

for SKU in SKU_range:
        results= results.append(pd.read_csv(PATH_RESULTS+'/SKU'+str(SKU)+
                                            '/Static'+model_type+'_MultiPolicy_'+sampling+z+'.csv'))
        
        
# for SKU in SKU_range:
#     for RO in RO_range:
#         results_RO = results_RO.append(pd.read_csv(PATH_RESULTS+'/SKU'+str(SKU)+
#                                                    '/Static'+model_type+'_'+sampling+'_e'+str(RO)+'.csv'))
        
#results_local = pd.read_csv(PATH_RESULTS+'/StaticWeightedSAA_MultiPolicy_local_summary.csv')
#results_global = pd.read_csv(PATH_RESULTS+'/StaticWeightedSAA_MultiPolicy_global_summary.csv')

In [ ]:
results

In [ ]:
results.to_csv(
    path_or_buf=PATH_RESULTS+'/StaticWeightedSAA_MultiPolicy_global_z.csv', 
    sep=','
)

In [ ]:
# results_RO.to_csv(
#                 path_or_buf=PATH_RESULTS+'/StaticWeightedSAA_MultiPolicy_RO_local_summary.csv' , 
#                 sep=','
#             )

In [ ]:
results_RO_agg = results_RO.groupby(['SKU', 'e_std', 'SL', 'T_horizon_rolling']).agg({'cost': sum}).groupby(
   ['SKU', 'e_std', 'SL']).agg({'cost': min}).reset_index()

results_RO_agg_best = results_RO.groupby(
    ['SKU', 'e_std', 'SL', 'T_horizon_rolling']).agg({'cost': sum}).groupby(
    ['SKU', 'T_horizon_rolling', 'SL']).agg({'cost': min}).groupby(
    ['SKU', 'SL']).agg({'cost': min}).reset_index()

results_local_agg = results_local.groupby(['SKU', 'SL', 'T_horizon_rolling']).agg({'cost': sum}).groupby(
   ['SKU', 'SL']).agg({'cost': min}).reset_index()

results_global_agg = results_global.groupby(['SKU', 'SL', 'T_horizon_rolling']).agg({'cost': sum}).groupby(
   ['SKU', 'SL']).agg({'cost': min}).reset_index()

In [ ]:
results_RO_agg = results_RO_agg.pivot(index=['SKU', 'SL'], 
                     columns=['e_std'], 
                     values=['cost'])
results_RO_agg.columns = results_RO_agg.columns.to_flat_index()
results_RO_agg = results_RO_agg.reset_index()

In [ ]:
# Merge
result = results_RO_agg.merge(results_local_agg, on=['SKU', 'SL'])
result = result.merge(results_global_agg, on=['SKU', 'SL'], suffixes=['_local', '_global'])
result = result.merge(results_RO_agg_best, on=['SKU', 'SL'])
result=result.rename(columns={'cost': 'cost_bestRO'})
result = result.melt(id_vars=['SKU', 'SL'], var_name='model', value_name='cost')
result

In [ ]:
result.groupby(['SL', 'model']).agg({'cost': sum}).reset_index()

In [ ]:
# Import utils
import numpy as np
import pandas as pd
import seaborn as sns
import copy
import time
import datetime as dt
import pickle
import pyreadr
import json
from joblib import dump, load, Parallel, delayed
import joblib as joblib
import os
import itertools
import contextlib
from tqdm import tqdm

# Import ML tools
import sklearn
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Set params
params = {
   
    'SKU_range': range(1,460+1),
    'T_horizon_rolling_range': range(1,5+1),
    'T_horizon': 13,
    'n_jobs': 32,
    'k': 100,
    'PATH_DATA': '/home/fesc/Data',
    'PATH_PARAMS': '/home/fesc/Data/Params',
    'PATH_SAMPLES': '/home/fesc/Data/Samples',
    'PATH_RESULTS': '/home/fesc/Data/Results',
    'PATH_KERNELS': '/home/fesc/Data/Kernels'
}

In [ ]:
# Check if all weights where generated
SKUs = []
T_horizon_rollings = []
fileExists = []
timeLastModifieds = []

for SKU in params['SKU_range']:
    for T_horizon_rolling in params['T_horizon_rolling_range']:

        path = params['PATH_SAMPLES']+'/SKU'+str(SKU)+'/Static/Weights'+str(T_horizon_rolling)+'/weights_global_MQRX_ij.p'    
        fileCheck = os.path.exists(path)
        
        if fileCheck:
            timeLastModified = os.path.getmtime(path)
            timeLastModified = time.strftime('%Y-%m-%d-%H-%M-%S', time.gmtime(timeLastModified))
        else:
             timeLastModified = ""

        SKUs = SKUs + [SKU]
        T_horizon_rollings = T_horizon_rollings + [T_horizon_rolling]
        fileExists = fileExists + [fileCheck]
        timeLastModifieds = timeLastModifieds + [timeLastModified]

        
checkWeights = pd.DataFrame({
    'SKU': SKUs,
    'T_horizon_rolling': T_horizon_rolling,
    'weightsExist': fileExists,
    'timeLastModified': timeLastModifieds
})

In [ ]:
checkWeights.loc[checkWeights.weightsExist]

In [ ]:
checkWeights.sort_values('timeLastModified')

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results'
model_type = 'WeightedSAA' # Use 'SAA' or 'WeightedSAA'
sampling = 'global' # Use 'global' or 'local'


# SKU range
SKU_range = range(1,460+1)




fileExists = []
SKUs = []

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    fileExists = fileExists + [os.path.exists(PATH_RESULTS+'/SKU'+str(SKU)+'/StaticWeightedSAA_global_robust_e6_20220630.csv')]
    SKUs = SKUs + [SKU]
        
        
        
ResultsCheck = pd.DataFrame({
    'SKU': SKUs,
    'fileExists': fileExists
})




In [ ]:
ResultsCheck.loc[ResultsCheck.fileExists == False]

In [ ]:
len(ResultsCheck.loc[ResultsCheck.fileExists == False])

In [ ]:
PATH_RESULTS = '/home/fesc/Data/Results'
model_type = 'WeightedSAA' # Use 'SAA' or 'WeightedSAA'
sampling = 'global' # Use 'global' or 'local'


# SKU range
SKU_range = range(1,460+1)




results = pd.DataFrame()

# For all SKUs
for SKU in SKU_range:
      
    # Check if results exist   
    fileExists = fileExists + [os.path.exists(PATH_RESULTS+'/SKU'+str(SKU)+'/StaticWeightedSAA_global_robust_e1_20220630.csv')]
    
    if fileExists:
        results= results.append(pd.read_csv(PATH_RESULTS+'/SKU'+str(SKU)+'/StaticWeightedSAA_global_robust_e1_20220630.csv'))
        
        
        





In [ ]:
results.to_csv(
        path_or_buf=PATH_RESULTS+'/StaticWeightedSAA_global_robust_e1_20220630.csv', 
        sep=','
    )